<h2 align=center>Generate Synthetic Images with DCGANs in Keras</h2>

## Task 1: Project Overview and Import Libraries

In [ ]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import numpy as np
import plot_utils
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython import display
print('Tensorflow version:', tf.__version__)

## Task 2: Load and Preprocess the Data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
# we need to normalize the pixel values which range from 0 to 255 to 0 to 1
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

In [ ]:
plt.figure(figsize = (10,10))
# creating a 5 X 5 grid for each image
# we will be showing 25 images from the training set
# binary because they are greyscale images so no colour channel, just black/white images
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap = plt.cm.binary)
plt.show()

## Task 3: Create Batches of Training Data

In [ ]:
batch_size = 32
# shuffle the example with a size of 1000. the datset fills this buffer with 1000 elements, then
# randomly samples from this buffer, then replaces the selected elements with new elements
# This dataset fills a buffer with buffer_size elements, 
#then randomly samples elements from this buffer, replacing the selected elements with new elements.
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000)
# pre-fetch elements from the dataset with buffer size = 1
# drop_remaider = True so that tensorflow ignores the last batch to get a full shape propagation
#Combines consecutive elements of this dataset into batches.
dataset = dataset.batch(batch_size, drop_remainder = True).prefetch(1)
#Creates a Dataset that prefetches elements from this dataset

## Task 4: Build the Generator Network for DCGAN

![GAN](DCGAN.png)

**Generator** is an upsampling network with fractionally-strided convolutions

**Discriminator** is a convolutional network

Architecture guidelines for stable Deep Convolutional GANs:

(The points of difference between GANS and DCGANS):

- Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator).
- Use batchnorm in both the generator and the discriminator.
- Remove fully connected hidden layers for deeper architectures.
- Use ReLU(or SELU) activation in generator for all layers expect for the output, which uses Tanh.
- Use LeakyReLU activation in the discriminator for all layers(except for output layer where we will be using sigmoid activation function to perform binary classfication).

[Source](https://arxiv.org/pdf/1511.06434.pdf): Radford et al, "*Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks*", ICLR 2016

We wanted to sample from complex, high dimensional distribution to generate the synthetic images but there is no direct way of doing that. So we will be making a transformation from the random Guassian noise distribution to the training images. We want the model to learn a transformation from this simple distribtuion(Guassian noise) to the training distribution(set).

Discrimator is just like a binary CNN image classifier which tries to identify whether an image is real or fake.

It would be done using a two player game theoretic formulation using where te 2 players are the generator and the discrimator networks, they are going to be trained jointly using a mini-max game theoretic formulation.

We are going to perform gradient descent on both the generator and discrimator where the objective function of the generator will be to try to prove the discriminator wrong.

 Execute only 1 of the below 2 cells

In [ ]:
# we can set any value for no. of features ranging from 30 to much higher depeding on compute
# power etc
# we are going to upsample from the seed(random noise) until we get to the image size
num_features = 100
# input shape is the no of features
generator = keras.models.Sequential([
    keras.layers.Dense(7 * 7 * 128, input_shape = [num_features]),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, (5,5), (2,2), padding = 'same', activation = 'selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, (5,5), (2,2), padding = 'same', activation = 'tanh')
])
# we can also start with 7 * 7 * 256 instead of 128 to improve performance
# keras.layers.Dense(7 * 7 * 256, input_shape = [num_features]),
# keras.layers.Reshape([7, 7, 256]),
# then we can duplicate the conv2D layer and instead of having 64, we can have 128
# keras.layers.Conv2DTranspose(128, (5,5), (1,1), padding = 'same', activation = 'selu'),
# keras.layers.BatchNormalization(),

In [ ]:
# Something like this:
num_features = 100
# input shape is the no of features
generator = keras.models.Sequential([
    keras.layers.Dense(7 * 7 * 256, input_shape = [num_features]),
    keras.layers.Reshape([7, 7, 256]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(128, (5,5), (1,1), padding = 'same', activation = 'selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, (5,5), (2,2), padding = 'same', activation = 'selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, (5,5), (2,2), padding = 'same', activation = 'tanh')
])

In [ ]:
# seed or a vector of random noise we are going to pass to the generator to see the o/p without
# the network being trained
noise = tf.random.normal(shape = [1, num_features])
generated_image = generator(noise, training = False)
plot_utils.show(generated_image, 1)
# this will show image of random noise, once the generator model has been trained, it will show
# images from the training_set(by transforming this random noise)

## Task 5: Build the Discriminator Network for DCGAN

Execute only 1 of the below 2 cells

In [ ]:
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, (5,5), (2,2), padding = 'same', input_shape = [28, 28, 1]),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(128, (5,5), (2,2), padding = 'same'),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation = 'sigmoid')
])
# we can repeat these layers, change 128 to 256 in this layer, stride to 1
# keras.layers.Conv2D(256, (5,5), (1,1), padding = 'same'),
# keras.layers.LeakyReLU(0.2),
# keras.layers.Dropout(0.3),

In [ ]:
# something like this
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, (5,5), (2,2), padding = 'same', input_shape = [28, 28, 1]),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(128, (5,5), (2,2), padding = 'same'),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(256, (5,5), (1,1), padding = 'same'),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
# decision will be a no b/w 0 and 1, if it is < 0.5, it means fake image, else real image
decision = discriminator(generated_image)
print(decision)

## Task 6: Compile the Deep Convolutional Generative Adversarial Network (DCGAN)

In [ ]:
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')
discriminator.trainable = False
gan = keras.models.Sequential([generator, discriminator])
gan.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')

## Task 7: Define Training Procedure

In [ ]:
# this creates the seed image
seed = tf.random.normal(shape = [batch_size, num_features])

In [ ]:
def train_dcgan(gan, dataset, batch_size, num_features, epochs = 5):
    generator, discriminator = gan.layers
    for epoch in tqdm(range(epochs)):
        print("Epoch {}/{}".format(epoch + 1, epochs))
        for X_batch in dataset:
            noise = tf.random.normal(shape = [batch_size, num_features])
            generated_images = generator(noise)
            # concatenate list of generated images with the list of real images
            X_fake_and_real = tf.concat([generated_images, X_batch], axis = 0)
            # labels for fake image = 0, for real mages = 1
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        display.clear_output(wait = True)
        generate_and_save_images(generator, epoch + 1, seed)
    
    display.clear_output(wait = True)
    generate_and_save_images(generator, epochs, seed)

In [ ]:
## Source https://www.tensorflow.org/tutorials/generative/dcgan#create_a_gif
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(10,10))
    
    for i in range(25):
        plt.subplot(5, 5, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='binary')
        plt.axis('off')
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

## Task 8: Train DCGAN

In [ ]:
x_train_dcgan = x_train.reshape(-1, 28, 28, 1) * 2. - 1.

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(x_train_dcgan).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder = True).prefetch(1)

In [ ]:
%%time
train_dcgan(gan, dataset, batch_size, num_features, epochs = 10)

## Task 9: Generate Synthetic Images with DCGAN

In [ ]:
noise = tf.random.normal(shape = [batch_size, num_features])
generated_images = generator(noise)
plot_utils.show(generated_images, 8)

In [ ]:
## Source: https://www.tensorflow.org/tutorials/generative/dcgan#create_a_gif

import imageio
import glob

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('image*.png')
    filenames = sorted(filenames)
    last = -1
    for i,filename in enumerate(filenames):
        frame = 2*(i**0.5)
        if round(frame) > round(last):
            last = frame
        else:
            continue
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

import IPython
display.Image(filename=anim_file)

We can make this model better by training it with larger no of epochs, say 50 or 100
we can also make it better by increasing the no of layers in both the generator and discriminator